In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.listdir('/kaggle/input/clone-mmdetection-feng/mmdetection_feng/wheat_detection')
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
import torch
print(torch.cuda.is_available())
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ../input/mmcvwhl/addict-2.2.1-py3-none-any.whl
!pip install ../input/mmdetection20-5-13/mmcv-0.5.1-cp37-cp37m-linux_x86_64.whl
!pip install ../input/mmdetection20-5-13/terminal-0.4.0-py3-none-any.whl
!pip install ../input/mmdetection20-5-13/terminaltables-3.1.0-py3-none-any.whl

#### Copy the MMDetection framework to the writeable directoy.

In [ ]:
!ls -l /kaggle/input/clone-mmdetection-feng/mmdetection_feng/mmdet/models/detectors

In [ ]:
!cat /kaggle/input/clone-mmdetection-feng/mmdetection_feng/mmdet/models/detectors/base.py

In [ ]:
!cp -r /kaggle/input/clone-mmdetection-feng/mmdetection_feng/ /kaggle/working/mmdetection_feng

In [ ]:
os.listdir('/kaggle/input/clone-mmdetection-feng/mmdetection_feng')
os.listdir('.')

In [ ]:
os.listdir('/kaggle/input/clone-mmdetection-feng/mmdetection_feng/configs/wheat_detection/')
os.listdir('/kaggle/input/clone-mmdetection-feng/mmdetection_feng/mmdet/datasets')

#### compile the coco toolbox

In [ ]:
!cp -r /kaggle/input/mmdetection20-5-13/cocoapi/cocoapi .

In [ ]:
cd /kaggle/working/cocoapi/PythonAPI

In [ ]:
os.listdir('/kaggle/working/')

In [ ]:
!make

In [ ]:
!make install

In [ ]:
import pycocotools

In [ ]:
cd /kaggle/working/mmdetection_feng

In [ ]:
!pip install -v -e.

In [ ]:
cd /kaggle/working/mmdetection_feng

In [ ]:
# !cat ./mmdet/models/detectors/base.py

In [ ]:
import sys
sys.path.append('/kaggle/working/mmdetection_feng') # To find local version

In [ ]:
# os.listdir()

In [ ]:
# !ls -l /kaggle/working/mmdetection_feng/mmdet/models/detectors

In [ ]:
# !cat /kaggle/working/mmdetection_feng/mmdet/models/detectors/base.py

In [ ]:
import mmdet

In [ ]:
import time

from mmdet.apis import init_detector, inference_detector
from tqdm import tqdm
import os
import cv2

import pandas as pd

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)


In [ ]:
# def submission_test(config_file, checkpoint_file):

#     model = init_detector(config_file, checkpoint_file, device='cuda:0')

#     results = []
#     test_path = '/kaggle/input/global-wheat-detection/test'
#     bar = tqdm(os.listdir(test_path))
#     for file in bar:
#         img = f'{test_path}/{file}'
#         result = inference_detector(model, img)
#         result = [_ for _ in result if len(_) != 0]
#         boxes = []
#         scores = []

#         image = cv2.imread(img)
#         for class_result in result:
#             for item in class_result:
#                 box = item[:4]
#                 x1, y1, x2, y2 = box
#                 score = item[4]
#                 boxes.append([x1, y1, x2 - x1, y2 - y1])
#                 scores.append(float(score))

#         cur_image_result = {
#             'image_id': file[:-4],
#             'PredictionString': format_prediction_string(boxes, scores)
#         }
#         results.append(cur_image_result)


#     test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
#     return test_df

In [ ]:

def submission_test(config_file, checkpoint_file):

    model = init_detector(config_file, checkpoint_file, device='cuda:0')

    results = []
    test_path = '/kaggle/input/global-wheat-detection/test'
    # test_path = '/home/fengyouliang/datasets/WHD/test'

    bar = tqdm(os.listdir(test_path))
    for file in bar:
        img = f'{test_path}/{file}'
        result = inference_detector(model, img)
        if isinstance(result, tuple):
            bbox_result, segm_result = result
            if isinstance(segm_result, tuple):
                segm_result = segm_result[0]  # ms rcnn
        else:
            bbox_result, segm_result = result, None
        result = [_ for _ in bbox_result if len(_) != 0]
        boxes = []
        scores = []

        for class_result in result:
            for item in class_result:
                box = item[:4]
                x1, y1, x2, y2 = box
                score = item[4]
                boxes.append([x1, y1, x2 - x1, y2 - y1])
                scores.append(float(score))

        cur_image_result = {
            'image_id': file[:-4],
            'PredictionString': format_prediction_string(boxes, scores)
        }
        results.append(cur_image_result)

    test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
    return test_df


## Inference

In [ ]:
import os

path = '/kaggle/input/maskrcnn/'
os.listdir(path)

In [ ]:
root_path = path

config_file = f'{root_path}/maskrcnn.py'
checkpoint_file = f'{root_path}/epoch_12.pth'

# with open(config_file, 'r') as f:
#     py = f.readlines()
    
import os
os.path.isfile(config_file), os.path.isfile(checkpoint_file)

In [ ]:
# for _ in py:
#     print(_[:-1])

In [ ]:
test_df = submission_test(config_file, checkpoint_file)

# save_path = '/kaggle/working'
#     if not os.path.exists(save_path):
#         os.makedirs(save_path)
#     test_df.to_csv(f'{save_path}/submission.csv', index=False)

In [ ]:
! rm -rf /kaggle/working

In [ ]:
test_df.to_csv('/kaggle/working/submission.csv', index=False)